In [1]:
from opensea import assets,bundles,common,events
import json

# JSON keys in API CALL

In [46]:
asset = assets.get_assets(collection="cryptokitties",limit=1,verified_only=True)
print("First index asset keys")
asset[0].get_json().keys()

First index asset keys


dict_keys(['id', 'token_id', 'num_sales', 'background_color', 'image_url', 'image_preview_url', 'image_thumbnail_url', 'image_original_url', 'animation_url', 'animation_original_url', 'name', 'description', 'external_link', 'asset_contract', 'permalink', 'collection', 'decimals', 'token_metadata', 'owner', 'sell_orders', 'creator', 'traits', 'last_sale', 'top_bid', 'listing_date', 'is_presale', 'transfer_fee_payment_token', 'transfer_fee', 'related_assets', 'orders', 'auctions', 'supports_wyvern', 'top_ownerships', 'ownership', 'highest_buyer_commitment'])

In [47]:
# In depth look at json, keys such as collection has more subkeys
json_asset = json.dumps(asset[0].get_json(), indent = 4, sort_keys=True) 
print(json_asset)

{
    "animation_original_url": null,
    "animation_url": null,
    "asset_contract": {
        "address": "0x06012c8cf97bead5deae237070f9587f8e7a266d",
        "asset_contract_type": "non-fungible",
        "buyer_fee_basis_points": 0,
        "created_date": "2018-01-23T04:51:38.832339",
        "default_to_fiat": false,
        "description": "CryptoKitties is a game centered around breedable, collectible, and oh-so-adorable creatures we call CryptoKitties! Each cat is one-of-a-kind and 100% owned by you; it cannot be replicated, taken away, or destroyed.",
        "dev_buyer_fee_basis_points": 0,
        "dev_seller_fee_basis_points": 0,
        "external_link": "https://www.cryptokitties.co/",
        "image_url": "https://lh3.googleusercontent.com/C272ZRW1RGGef9vKMePFSCeKc1Lw6U40wl9ofNVxzUxFdj84hH9xJRQNf-7wgs7W8qw8RWe-1ybKp-VKuU5D-tg=s60",
        "name": "CryptoKitties",
        "nft_version": "1.0",
        "only_proxied_transfers": false,
        "opensea_buyer_fee_basis_poin

We can copy this and paste into http://jsonviewer.stack.hu/ to interactively see our variables we can use

If you take a look at opensea > models > asset.py 

``` Python
class Asset:
    def __init__(self, json_data):
        
        """
        Put useful information from json_data into their own variables under Asset class.

        :param json_data: json object returned by the opensea-api
        :type json_data: dict
        """
        # ASSET DETAILS
        self.name = json_data["name"]
        self.description = json_data["description"]
        self.token_id = json_data["token_id"]
        self.asset_url = json_data["permalink"]
        self.image_url = json_data["image_url"]
```

If we want to add say the created_date of the crypto we just need to add 
``` Python
self.created_date = json_data["created_date"]
```
and we will be able to call
``` Python
print(asset.created_date)
```

# A look into the Images from OpenSea

In [35]:
# This will return a list of assets which you can iterate and get the needed data
# Note we will ignore bundles (Collection of NFTs sold together)
from PIL import Image
# load the image
import requests
from io import BytesIO

asset_list = assets.get_assets(collection="cryptokitties",limit=3,verified_only=True)

for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list
    url = asset.image_url
    print(url)
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    # summarize some details about the image
    print(img.format)
    print(img.mode)
    print(img.size)
    # show the image
    img.show()

https://lh3.googleusercontent.com/wzrRAgkq0gwq1_jg_RN3_hW1eA8CwPefobfyjQP5QCPxBVOoZX5BY4suaP1htwvVlGE5rR4-z-duWJfQJ8ye-g
PNG
RGBA
(512, 512)
https://lh3.googleusercontent.com/2gizLG71E-qiymsqRWACPhPDKJznZPMzFzVfyf70FQNt0iuP_A949WkmHn4JcQdYepUGWdIY_4UwZJ1_TmVeq1Vv7bFnjbxWa600nMs
PNG
RGBA
(512, 512)
https://storage.opensea.io/0x06012c8cf97bead5deae237070f9587f8e7a266d/305363-1555102810.png
PNG
RGBA
(2000, 2000)


Note that we have images with different resolutions, this is something we will need to deal with, prehaps a simple rescale (svd?)

In [40]:
# This will return a list of assets which you can iterate and get the needed data
# Note we will ignore bundles (Collection of NFTs sold together)

asset_list = assets.get_assets(limit=5,verified_only=True)

for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list

    print(asset.owner.address)
    print(asset.owner.profile_img_url)
    print(asset.owner.username)
    print(asset.owner.config)

    print(asset.name)
    print(asset.description)
    print(asset.asset_url)
    print(asset.image_url)
    print(asset.get_floor_price()) # Floor price of the collection

0xc8db25e7d49ae0c4f4a63b98203fce908b052cf4
https://storage.googleapis.com/opensea-static/opensea-profile/2.png
Offerzaccept

Rare Whixtensions
Oh hi! I'm Rare Whixtensions. I want to live in a world where people believe that one day cats will rule this planet. I would give it all up for a Chanel purse. Let's pawty!
https://opensea.io/assets/0x06012c8cf97bead5deae237070f9587f8e7a266d/126980
https://lh3.googleusercontent.com/wzrRAgkq0gwq1_jg_RN3_hW1eA8CwPefobfyjQP5QCPxBVOoZX5BY4suaP1htwvVlGE5rR4-z-duWJfQJ8ye-g
0.0
0x5a8fbfc2b797b802a2ebf4ea1cc297d2a825b791
https://storage.googleapis.com/opensea-static/opensea-profile/1.png
None

Ledger Legend Card

https://opensea.io/assets/0xde083e40fe84835cbbd6c69f6595cae1e85551dc/136
https://storage.googleapis.com/opensea-prod.appspot.com/0xde083e40fe84835cbbd6c69f6595cae1e85551dc/136.png
5.0
0x5a8fbfc2b797b802a2ebf4ea1cc297d2a825b791
https://storage.googleapis.com/opensea-static/opensea-profile/1.png
None

Ledger Legend Card

https://opensea.io/asset

# A look into querying 1 NFT collection

In [6]:

# This will return a list of assets which you can iterate and get the needed data
# Note we will ignore bundles (Collection of NFTs sold together)

# Order by sale_price to get order by last total_price

asset_list = assets.get_assets(collection="cryptokitties",limit=1, verified_only=True, token_ids=896775)

for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list
    print("i:", i)
    # print(asset.owner.address)    

    print(asset.name)
    print(asset.description)
    print(asset.asset_url)
    # print(asset.image_url)
    print("price floor", asset.get_floor_price()) # Floor price of the collection

    


i: 0
Dragon
Salutations. I'm Dragon. I bit Rebecca Black when I was younger, but those days are gone. Believe it or not, spying on the neighbours is seriously exhilarating. You should try it some time. Let's connect on ICQ.
https://opensea.io/assets/0x06012c8cf97bead5deae237070f9587f8e7a266d/896775
price floor 0.0


In [4]:
# In depth look at json, keys such as collection has more subkeys
json_asset = json.dumps(asset_list[0].get_json(), indent = 4, sort_keys=True) 
print(json_asset)

{
    "animation_original_url": null,
    "animation_url": null,
    "asset_contract": {
        "address": "0x06012c8cf97bead5deae237070f9587f8e7a266d",
        "asset_contract_type": "non-fungible",
        "buyer_fee_basis_points": 0,
        "created_date": "2018-01-23T04:51:38.832339",
        "default_to_fiat": false,
        "description": "CryptoKitties is a game centered around breedable, collectible, and oh-so-adorable creatures we call CryptoKitties! Each cat is one-of-a-kind and 100% owned by you; it cannot be replicated, taken away, or destroyed.",
        "dev_buyer_fee_basis_points": 0,
        "dev_seller_fee_basis_points": 0,
        "external_link": "https://www.cryptokitties.co/",
        "image_url": "https://lh3.googleusercontent.com/C272ZRW1RGGef9vKMePFSCeKc1Lw6U40wl9ofNVxzUxFdj84hH9xJRQNf-7wgs7W8qw8RWe-1ybKp-VKuU5D-tg=s60",
        "name": "CryptoKitties",
        "nft_version": "1.0",
        "only_proxied_transfers": false,
        "opensea_buyer_fee_basis_poin

# Looking at NFT EVENT transactions

In [5]:
asset_list = events.get_events(event_type='successful',asset_contract_address="0x06012c8cf97BEaD5deAe237070F9587f8E7A266d", token_id=896775)
asset_list


{'asset_events': [{'approved_account': None,
   'asset': {'id': 1314985,
    'token_id': '896775',
    'num_sales': 2,
    'background_color': 'efe1da',
    'image_url': 'https://lh3.googleusercontent.com/6eGDmoLqiLgsFPMBji-rzk6I-Xlr9wt8sxbbLH18ONzEVY3-GbsdKo08_5xsBYtEJKCnBj5HyBFL6Y3tI8gRWBzxTw',
    'image_preview_url': 'https://lh3.googleusercontent.com/6eGDmoLqiLgsFPMBji-rzk6I-Xlr9wt8sxbbLH18ONzEVY3-GbsdKo08_5xsBYtEJKCnBj5HyBFL6Y3tI8gRWBzxTw=s250',
    'image_thumbnail_url': 'https://lh3.googleusercontent.com/6eGDmoLqiLgsFPMBji-rzk6I-Xlr9wt8sxbbLH18ONzEVY3-GbsdKo08_5xsBYtEJKCnBj5HyBFL6Y3tI8gRWBzxTw=s128',
    'image_original_url': 'https://img.cryptokitties.co/0x06012c8cf97bead5deae237070f9587f8e7a266d/896775.png',
    'animation_url': None,
    'animation_original_url': None,
    'name': 'Dragon',
    'description': "Salutations. I'm Dragon. I bit Rebecca Black when I was younger, but those days are gone. Believe it or not, spying on the neighbours is seriously exhilarating. You sh

In [13]:
asset_list["asset_events"][0]

{'approved_account': None,
 'asset': {'id': 1314985,
  'token_id': '896775',
  'num_sales': 2,
  'background_color': 'efe1da',
  'image_url': 'https://lh3.googleusercontent.com/6eGDmoLqiLgsFPMBji-rzk6I-Xlr9wt8sxbbLH18ONzEVY3-GbsdKo08_5xsBYtEJKCnBj5HyBFL6Y3tI8gRWBzxTw',
  'image_preview_url': 'https://lh3.googleusercontent.com/6eGDmoLqiLgsFPMBji-rzk6I-Xlr9wt8sxbbLH18ONzEVY3-GbsdKo08_5xsBYtEJKCnBj5HyBFL6Y3tI8gRWBzxTw=s250',
  'image_thumbnail_url': 'https://lh3.googleusercontent.com/6eGDmoLqiLgsFPMBji-rzk6I-Xlr9wt8sxbbLH18ONzEVY3-GbsdKo08_5xsBYtEJKCnBj5HyBFL6Y3tI8gRWBzxTw=s128',
  'image_original_url': 'https://img.cryptokitties.co/0x06012c8cf97bead5deae237070f9587f8e7a266d/896775.png',
  'animation_url': None,
  'animation_original_url': None,
  'name': 'Dragon',
  'description': "Salutations. I'm Dragon. I bit Rebecca Black when I was younger, but those days are gone. Believe it or not, spying on the neighbours is seriously exhilarating. You should try it some time. Let's connect on IC

# Running through to see a how many kitties are currently on sale, or have a last_price

In [8]:
counter = 0 
for i in range(0, 400):
    asset_list = assets.get_assets(collection="cryptokitties",limit=30,verified_only=True,order_by="token_id",order_direction="asc",token_ids=list(range((i*50)+1, (i*50)+31)))
    print("i:", i)
    try:
        for j in range(len(asset_list)):
            asset = asset_list[j] # Get the first asset obejct from the list
            print(asset.name, asset.last_sale/1000000000000000000)
            counter += 1
            print(counter)
    except:
        pass

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
i: 27
i: 28
i: 29
i: 30
i: 31
i: 32
i: 33
i: 34
i: 35
i: 36
i: 37
i: 38
i: 39
i: 40
i: 41
i: 42
i: 43
i: 44
i: 45
i: 46
i: 47
i: 48
i: 49
i: 50
i: 51
i: 52
i: 53
i: 54
i: 55
i: 56
i: 57
i: 58
i: 59
i: 60
i: 61
i: 62
i: 63
i: 64
i: 65
i: 66
i: 67
i: 68
i: 69
i: 70
i: 71
i: 72
i: 73
i: 74
i: 75
i: 76
i: 77
i: 78
i: 79
i: 80
i: 81
i: 82
i: 83
i: 84
i: 85
i: 86
i: 87
i: 88
i: 89
i: 90
i: 91
i: 92
i: 93
i: 94
i: 95
i: 96
i: 97
i: 98
i: 99
i: 100
i: 101
i: 102
i: 103
i: 104
i: 105
i: 106
i: 107
i: 108
i: 109
i: 110
i: 111
i: 112
i: 113
i: 114
i: 115
i: 116
i: 117
i: 118
i: 119
i: 120
i: 121
i: 122
i: 123
i: 124
i: 125
i: 126
i: 127
i: 128
i: 129
i: 130
i: 131
i: 132
i: 133
i: 134
i: 135
i: 136
i: 137
i: 138
i: 139
i: 140
i: 141
i: 142
i: 143
i: 144
i: 145
i: 146
i: 147
i: 148
i: 149
i: 150
i: 151
i: 152
i: 153
i: 154
i: 155
i: 156
i: 157
i: 1

From cffi callback <function _verify_callback at 0x000001DF79F1FEE0>:
Traceback (most recent call last):
  File "C:\Users\cawin\anaconda3\lib\site-packages\OpenSSL\SSL.py", line 311, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='api.opensea.io', port=443): Max retries exceeded with url: /api/v1/assets?owner=&order_by=token_id&order_direction=asc&offset=0&limit=30&collection=cryptokitties&token_ids= (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [9]:
asset_list = assets.get_assets(collection="cryptokitties",limit=20, verified_only=True)

for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list
    print("i:", i)
    # print(asset.owner.address)    

    print(asset.name)
    print(asset.description)
    print(asset.asset_url)
    # print(asset.image_url)
    print(asset.token_id)
    print("price floor", asset.get_floor_price()) # Floor price of the collection

TypeError: object of type 'Response' has no len()